In [1]:
import sys
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime

%load_ext autoreload
%autoreload 2


In [3]:
def prepare_parquets(logpath):
    logpath = Path(logpath)
    logger_set = LoggerSet(logpath, overwrite_ok=False)
    logger_set.export_to_parquet()

def get_frame_idx(camera_df, angular_speed_control_df, time_offset_ms=57):
    camera_df = camera_df.copy()
    angular_speed_control_df = angular_speed_control_df.copy()
        
    camera_df['time_expected'] = camera_df['time'] + pd.to_timedelta(time_offset_ms, unit='ms')
    angular_speed_control_df['frame_idx'] = camera_df['time_expected'].searchsorted(angular_speed_control_df['time_AngularSpeedControl'], side='right')

    return angular_speed_control_df

def stack_frames(gen: Iterable[Tuple[int, np.ndarray]]):
    frames = []
    for idx, img in gen:
        frames.append(img)
    return np.stack(frames, axis=0)

def prep_for_session(logpath, time_offset_ms=100):

    logpath = Path(logpath)
    prepare_parquets(logpath)
    camera_df =  pd.read_parquet(logpath/'PicameraV2.parquet')
    control_df = pd.read_parquet(logpath/'AngularSpeedControlV2.parquet')

    control_df = get_frame_idx(camera_df, control_df, time_offset_ms=time_offset_ms)

    stacked_frames = stack_frames(get_frame_iterator(logpath/"PicameraV2/video"))

    return stacked_frames, control_df

In [4]:
def concatenate_multiple_sessions(data_of_sessions: List[Tuple[np.ndarray, pd.DataFrame]]):
    """
    MODIFY THE DATAFRAME *IN PLACE* as a side effect
    """
    frame_set_list = []
    df_list = []
    total_frames = 0
    for frames, df in data_of_sessions:
        df['total_frame_idx'] = df['frame_idx'] + total_frames

        total_frames += len(frames)
        df_list.append(df)
        frame_set_list.append(frames)

    return np.concatenate(frame_set_list, axis=0), pd.concat(df_list)

def load_multiple_session(session_paths):
    frames, dfs = [], []
    for logpath in tqdm(session_paths):
        frame, df = prep_for_session(logpath)
        frames.append(frame)
        dfs.append(df)
    
    return concatenate_multiple_sessions(zip(frames, dfs))

def prepare_to_tensorflow(frames, df):

    df = df.query('speed != 0')

    frames_reindexed = frames[df['frame_idx']]


    return frames_reindexed, df[['angular_velocity', 'left', 'right', 'speed']].values/100

frames, df = load_multiple_session(['./data/2024-05-31 20:28:39.871348/', './data/2024-05-31 20:44:54.302121/'])
data = prepare_to_tensorflow(frames, df)


100%|██████████| 2/2 [00:02<00:00,  1.36s/it]


In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imggen = ImageDataGenerator(
        rescale=1/255
    ).flow(*data)

2024-06-01 21:16:29.403258: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 21:16:29.973900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:

def get_model(lr=.1):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3
    
    model = keras.Sequential([
        keras.layers.InputLayer(image_shape), 
        keras.layers.Conv2D(16, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(32, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'), 
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(4), 
    ])

    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE'])

    return model 

model = get_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 112, 16)    │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 54, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 27, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 12, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 10, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,076 (590.14 KB)

 Trainable params: 151,076 (590.14 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model = get_model(3e-4)

model.fit(
    imggen, 
    epochs=100, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
        use_tensorboard('training'), 
        ] 
    )

Epoch 1/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - MAE: 0.3298 - loss: 0.1030
Epoch 2/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2958 - loss: 0.0857
Epoch 3/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2863 - loss: 0.0802
Epoch 4/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2785 - loss: 0.0754
Epoch 5/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2682 - loss: 0.0691
Epoch 6/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2612 - loss: 0.0650
Epoch 7/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2520 - loss: 0.0599
Epoch 8/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2492 - loss: 0.0581
Epoch 9/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2408 - loss: 0.0545
Epoch 10/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2342 - loss: 0.0516
Epoch 11/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2323 - loss: 0.0513
Epoch 12/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.2248 - l

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
import requests 
def find_lr(model, x, y=None, patience=10, start_lr=1e-6, epochs=100, verbose=1, **kwargs):
    
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: start_lr*10**(epoch/20)
    )
    
    history = model.fit(
        x, y, 
        epochs=epochs, 
        verbose=verbose,
        callbacks=[
            lr_schedule, tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience), 
            use_tensorboard('find_lr'), 
            end_epoch_notify(),
        ], 
    )
    return history

def use_tensorboard(key, main_dir='logs', append_time=True, histogram_freq=1, **kwargs):
    log_dir = f"{main_dir}/{key}" 
    if append_time: 
        log_dir += datetime.datetime.now().strftime("/%m-%d/%H:%M:%S")
    return tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=histogram_freq, **kwargs)

def tg_notify(msg): 
    token = "5077378964:AAGPQhx7oKRS4yLo9inFwE15CGdtwNaam8w"
    requests.get(f"https://api.telegram.org/bot{token}/sendMessage", dict(chat_id='988152989', text=msg))
    
def pretty_dict(data):
    r = ""
    for k, v in data.items():
        r += f"{k}: {v}\n"
    return r[:-1]

def end_epoch_notify():
    return tf.keras.callbacks.LambdaCallback(on_train_end=lambda log:tg_notify(pretty_dict(log)) )

In [ ]:
model = get_model(3e-4)

model.fit(
    imggen, 
    epochs=100, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
        use_tensorboard('training'), 
        ] 
    )

In [28]:
y = model.predict(imggen)

1147/1147 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [30]:
y*100

array([[ -4.9364743,  28.73624  ,  24.89859  ,  53.65413  ],
       [ -5.8969693,  59.78735  ,  87.06119  , 147.75409  ],
       [ -4.9364743,  28.73624  ,  24.89859  ,  53.65413  ],
       ...,
       [ -4.9364743,  28.73624  ,  24.898588 ,  53.65413  ],
       [  5.0087647,  68.15617  ,  42.51613  , 110.64633  ],
       [-11.533586 ,  26.239553 ,  46.75585  ,  74.65435  ]],
      dtype=float32)

In [19]:

#tensorboard --logdir src/training/logs --bind_all
find_lr(model, imggen)

Epoch 1/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - MAE: 0.6452 - loss: 0.2931 - learning_rate: 1.0000e-06
Epoch 2/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.5506 - loss: 0.2292 - learning_rate: 1.1220e-06
Epoch 3/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3999 - loss: 0.1392 - learning_rate: 1.2589e-06
Epoch 4/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3692 - loss: 0.1234 - learning_rate: 1.4125e-06
Epoch 5/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3559 - loss: 0.1163 - learning_rate: 1.5849e-06
Epoch 6/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3426 - loss: 0.1096 - learning_rate: 1.7783e-06
Epoch 7/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3303 - loss: 0.1034 - learning_rate: 1.9953e-06
Epoch 8/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3224 - loss: 0.0990 - learning_rate: 2.2387e-06
Epoch 9/100
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - MAE: 0.3170 - loss: 0.0954 - learning_rate: 2.

In [26]:
model.save('2Jun-pi.keras')